# Forecasting problems

To discuss a couple of forecasting problems and explore their solutions. It will be a long notebook

### Corporación Favorita Grocery Sales Forecasting

To forecast Daily Unit Sales by store and product for a horizon of 1 to 16 days for more than 210,000 time series. The contestants had access to 55 months of data for the 54 stores and 3901 products, as well as metadata for the stores and products, promotion indicators, holiday information, and oil prices.

### Recruit Restaurant Visitor Forecasting

To forecast Daily Restaurant Visits by restaurant for a horizon of 1 to 39 days. The contestants had access to 15 months of data for the 821 restaurants, as well as metadata for the restaurants, holiday information, and reservations for restaurant visits made at different times in advance.

### Walmart Recruiting - Store Sales Forecasting

To forecasts of Weekly Sales in dollars by department and store for a horizon of 1 to 39 weeks. The contestants had access to 33 months of data for the 45 stores and 81 departments, as well as metadata for the stores, holiday information, promotion indicators, weekly temperatures, fuel prices, consumer price index (CPI), and unemployment rate.

### Walmart Recruiting II: Sales in Stormy Weather

To accurately predict the sales of 111 potentially weather-sensitive products (like umbrellas, bread, and milk) around the time of major weather events at 45 of their retail locations.

### Rossmann Store Sales

To forecast Daily Sales in dollars by store for a horizon of 1 to 48 days. The contestants had access to 31 months of data for the 1115 stores, as well as metadata for the stores, promotion indicators, holiday information, weather information, and Google Trend statistics

### Wikipedia Web Traffic Forecasting

To forecast daily Wikipedia page visits for a horizon of 12 to 42 days. The contestants had access to 32 months of data for the page visits, as well as metadata for the Wikipedia pages.

# Corporación Favorita Grocery Sales Forecasting

https://www.kaggle.com/c/favorita-grocery-sales-forecasting

In [1]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

In [2]:
path = 'C:\\Users\\Yaolong-PC\\Downloads\\favorita-grocery-sales-forecasting\\input\\'

df_train = pd.read_csv(
    path+'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    path+"test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    path+"items.csv",
).set_index("item_nbr")

In [3]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


In [4]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [5]:
print("Preparing dataset...")

t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

Preparing dataset...


In [6]:
print("Training and predicting models...")

MAX_ROUNDS = 280
val_pred = []
test_pred = []
cate_vars = []

for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    model = CatBoostRegressor(
        iterations=MAX_ROUNDS, learning_rate=0.5,
        depth=4)
        
    model.fit(
        X_train, y_train[:, i],
        cat_features=cate_vars)
    
    val_pred.append(model.predict(X_val))
    test_pred.append(model.predict(X_test))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Training and predicting models...
Step 1
0:	learn: 0.7676832	total: 216ms	remaining: 1m
1:	learn: 0.6532021	total: 255ms	remaining: 35.5s
2:	learn: 0.6113439	total: 293ms	remaining: 27.1s
3:	learn: 0.5958956	total: 332ms	remaining: 22.9s
4:	learn: 0.5877026	total: 368ms	remaining: 20.2s
5:	learn: 0.5837279	total: 408ms	remaining: 18.6s
6:	learn: 0.5805493	total: 446ms	remaining: 17.4s
7:	learn: 0.5782491	total: 482ms	remaining: 16.4s
8:	learn: 0.5764057	total: 519ms	remaining: 15.6s
9:	learn: 0.5740496	total: 557ms	remaining: 15s
10:	learn: 0.5729414	total: 596ms	remaining: 14.6s
11:	learn: 0.5722020	total: 634ms	remaining: 14.2s
12:	learn: 0.5711638	total: 671ms	remaining: 13.8s
13:	learn: 0.5699681	total: 711ms	remaining: 13.5s
14:	learn: 0.5688545	total: 753ms	remaining: 13.3s
15:	learn: 0.5681261	total: 794ms	remaining: 13.1s
16:	learn: 0.5676064	total: 829ms	remaining: 12.8s
17:	learn: 0.5672531	total: 865ms	remaining: 12.6s
18:	learn: 0.5667647	total: 903ms	remaining: 12.4s
19:	l

158:	learn: 0.5512330	total: 6.09s	remaining: 4.64s
159:	learn: 0.5511779	total: 6.13s	remaining: 4.6s
160:	learn: 0.5511545	total: 6.16s	remaining: 4.55s
161:	learn: 0.5511226	total: 6.2s	remaining: 4.51s
162:	learn: 0.5510945	total: 6.24s	remaining: 4.48s
163:	learn: 0.5510698	total: 6.28s	remaining: 4.44s
164:	learn: 0.5510433	total: 6.31s	remaining: 4.4s
165:	learn: 0.5510169	total: 6.35s	remaining: 4.36s
166:	learn: 0.5509905	total: 6.38s	remaining: 4.32s
167:	learn: 0.5509562	total: 6.42s	remaining: 4.28s
168:	learn: 0.5509104	total: 6.46s	remaining: 4.24s
169:	learn: 0.5508796	total: 6.5s	remaining: 4.2s
170:	learn: 0.5508374	total: 6.54s	remaining: 4.17s
171:	learn: 0.5507942	total: 6.58s	remaining: 4.13s
172:	learn: 0.5507488	total: 6.61s	remaining: 4.09s
173:	learn: 0.5507167	total: 6.65s	remaining: 4.05s
174:	learn: 0.5506826	total: 6.68s	remaining: 4.01s
175:	learn: 0.5506360	total: 6.72s	remaining: 3.97s
176:	learn: 0.5506114	total: 6.76s	remaining: 3.93s
177:	learn: 0.550

37:	learn: 0.5784236	total: 1.4s	remaining: 8.93s
38:	learn: 0.5782119	total: 1.44s	remaining: 8.9s
39:	learn: 0.5780201	total: 1.48s	remaining: 8.86s
40:	learn: 0.5778968	total: 1.51s	remaining: 8.79s
41:	learn: 0.5777683	total: 1.55s	remaining: 8.76s
42:	learn: 0.5776027	total: 1.58s	remaining: 8.72s
43:	learn: 0.5774609	total: 1.61s	remaining: 8.66s
44:	learn: 0.5773091	total: 1.66s	remaining: 8.65s
45:	learn: 0.5772153	total: 1.69s	remaining: 8.62s
46:	learn: 0.5771232	total: 1.73s	remaining: 8.57s
47:	learn: 0.5770027	total: 1.76s	remaining: 8.52s
48:	learn: 0.5768416	total: 1.8s	remaining: 8.49s
49:	learn: 0.5767072	total: 1.84s	remaining: 8.45s
50:	learn: 0.5766149	total: 1.88s	remaining: 8.43s
51:	learn: 0.5765070	total: 1.92s	remaining: 8.4s
52:	learn: 0.5764394	total: 1.95s	remaining: 8.35s
53:	learn: 0.5763332	total: 1.98s	remaining: 8.31s
54:	learn: 0.5762437	total: 2.02s	remaining: 8.27s
55:	learn: 0.5761278	total: 2.06s	remaining: 8.24s
56:	learn: 0.5760432	total: 2.1s	re

198:	learn: 0.5697629	total: 7.18s	remaining: 2.92s
199:	learn: 0.5697431	total: 7.21s	remaining: 2.88s
200:	learn: 0.5697219	total: 7.25s	remaining: 2.85s
201:	learn: 0.5697031	total: 7.28s	remaining: 2.81s
202:	learn: 0.5696797	total: 7.32s	remaining: 2.78s
203:	learn: 0.5696544	total: 7.36s	remaining: 2.74s
204:	learn: 0.5696230	total: 7.39s	remaining: 2.71s
205:	learn: 0.5695925	total: 7.43s	remaining: 2.67s
206:	learn: 0.5695652	total: 7.47s	remaining: 2.63s
207:	learn: 0.5695404	total: 7.5s	remaining: 2.6s
208:	learn: 0.5695095	total: 7.54s	remaining: 2.56s
209:	learn: 0.5695011	total: 7.58s	remaining: 2.52s
210:	learn: 0.5694768	total: 7.61s	remaining: 2.49s
211:	learn: 0.5694552	total: 7.65s	remaining: 2.45s
212:	learn: 0.5694396	total: 7.68s	remaining: 2.42s
213:	learn: 0.5694171	total: 7.72s	remaining: 2.38s
214:	learn: 0.5693939	total: 7.76s	remaining: 2.35s
215:	learn: 0.5693691	total: 7.8s	remaining: 2.31s
216:	learn: 0.5693572	total: 7.83s	remaining: 2.27s
217:	learn: 0.5

76:	learn: 0.5786317	total: 2.82s	remaining: 7.44s
77:	learn: 0.5785370	total: 2.86s	remaining: 7.41s
78:	learn: 0.5784632	total: 2.9s	remaining: 7.37s
79:	learn: 0.5783807	total: 2.93s	remaining: 7.33s
80:	learn: 0.5783091	total: 2.97s	remaining: 7.29s
81:	learn: 0.5782381	total: 3s	remaining: 7.24s
82:	learn: 0.5781866	total: 3.03s	remaining: 7.2s
83:	learn: 0.5781258	total: 3.07s	remaining: 7.16s
84:	learn: 0.5780690	total: 3.1s	remaining: 7.12s
85:	learn: 0.5780124	total: 3.14s	remaining: 7.08s
86:	learn: 0.5779587	total: 3.17s	remaining: 7.04s
87:	learn: 0.5778687	total: 3.21s	remaining: 7s
88:	learn: 0.5778096	total: 3.25s	remaining: 6.97s
89:	learn: 0.5777343	total: 3.28s	remaining: 6.93s
90:	learn: 0.5776838	total: 3.32s	remaining: 6.9s
91:	learn: 0.5776308	total: 3.36s	remaining: 6.86s
92:	learn: 0.5775519	total: 3.39s	remaining: 6.83s
93:	learn: 0.5775010	total: 3.43s	remaining: 6.79s
94:	learn: 0.5774435	total: 3.46s	remaining: 6.75s
95:	learn: 0.5773902	total: 3.5s	remainin

...

63:	learn: 0.6057671	total: 2.3s	remaining: 7.76s
64:	learn: 0.6056606	total: 2.34s	remaining: 7.73s
65:	learn: 0.6055281	total: 2.37s	remaining: 7.69s
66:	learn: 0.6054518	total: 2.4s	remaining: 7.64s
67:	learn: 0.6053700	total: 2.44s	remaining: 7.6s
68:	learn: 0.6053147	total: 2.47s	remaining: 7.56s
69:	learn: 0.6052669	total: 2.51s	remaining: 7.52s
70:	learn: 0.6051829	total: 2.54s	remaining: 7.48s
71:	learn: 0.6050672	total: 2.58s	remaining: 7.44s
72:	learn: 0.6049094	total: 2.61s	remaining: 7.4s
73:	learn: 0.6047897	total: 2.65s	remaining: 7.37s
74:	learn: 0.6047073	total: 2.68s	remaining: 7.33s
75:	learn: 0.6046490	total: 2.72s	remaining: 7.3s
76:	learn: 0.6045883	total: 2.76s	remaining: 7.28s
77:	learn: 0.6045142	total: 2.79s	remaining: 7.23s
78:	learn: 0.6044399	total: 2.83s	remaining: 7.2s
79:	learn: 0.6043631	total: 2.86s	remaining: 7.15s
80:	learn: 0.6042927	total: 2.9s	remaining: 7.12s
81:	learn: 0.6042244	total: 2.93s	remaining: 7.09s
82:	learn: 0.6041655	total: 2.97s	rema

224:	learn: 0.5983968	total: 8.01s	remaining: 1.96s
225:	learn: 0.5983756	total: 8.04s	remaining: 1.92s
226:	learn: 0.5983522	total: 8.08s	remaining: 1.89s
227:	learn: 0.5982934	total: 8.11s	remaining: 1.85s
228:	learn: 0.5982699	total: 8.15s	remaining: 1.81s
229:	learn: 0.5982437	total: 8.19s	remaining: 1.78s
230:	learn: 0.5982195	total: 8.22s	remaining: 1.74s
231:	learn: 0.5981986	total: 8.26s	remaining: 1.71s
232:	learn: 0.5981725	total: 8.29s	remaining: 1.67s
233:	learn: 0.5981490	total: 8.33s	remaining: 1.64s
234:	learn: 0.5981205	total: 8.36s	remaining: 1.6s
235:	learn: 0.5980969	total: 8.4s	remaining: 1.56s
236:	learn: 0.5980794	total: 8.43s	remaining: 1.53s
237:	learn: 0.5980448	total: 8.47s	remaining: 1.49s
238:	learn: 0.5980268	total: 8.5s	remaining: 1.46s
239:	learn: 0.5980055	total: 8.54s	remaining: 1.42s
240:	learn: 0.5979682	total: 8.57s	remaining: 1.39s
241:	learn: 0.5979473	total: 8.61s	remaining: 1.35s
242:	learn: 0.5979256	total: 8.65s	remaining: 1.32s
243:	learn: 0.5

105:	learn: 0.5939112	total: 3.82s	remaining: 6.26s
106:	learn: 0.5938640	total: 3.85s	remaining: 6.22s
107:	learn: 0.5938175	total: 3.88s	remaining: 6.18s
108:	learn: 0.5937512	total: 3.92s	remaining: 6.15s
109:	learn: 0.5937032	total: 3.96s	remaining: 6.12s
110:	learn: 0.5936475	total: 3.99s	remaining: 6.08s
111:	learn: 0.5935972	total: 4.03s	remaining: 6.04s
112:	learn: 0.5935372	total: 4.07s	remaining: 6.01s
113:	learn: 0.5934972	total: 4.1s	remaining: 5.97s
114:	learn: 0.5934458	total: 4.13s	remaining: 5.93s
115:	learn: 0.5934237	total: 4.17s	remaining: 5.9s
116:	learn: 0.5933847	total: 4.21s	remaining: 5.86s
117:	learn: 0.5933538	total: 4.24s	remaining: 5.82s
118:	learn: 0.5933193	total: 4.27s	remaining: 5.78s
119:	learn: 0.5932839	total: 4.31s	remaining: 5.75s
120:	learn: 0.5932233	total: 4.35s	remaining: 5.72s
121:	learn: 0.5931747	total: 4.38s	remaining: 5.68s
122:	learn: 0.5931113	total: 4.42s	remaining: 5.64s
123:	learn: 0.5930812	total: 4.45s	remaining: 5.6s
124:	learn: 0.5

265:	learn: 0.5883879	total: 9.52s	remaining: 501ms
266:	learn: 0.5883670	total: 9.55s	remaining: 465ms
267:	learn: 0.5883452	total: 9.59s	remaining: 429ms
268:	learn: 0.5883251	total: 9.63s	remaining: 394ms
269:	learn: 0.5883046	total: 9.66s	remaining: 358ms
270:	learn: 0.5882886	total: 9.7s	remaining: 322ms
271:	learn: 0.5882701	total: 9.73s	remaining: 286ms
272:	learn: 0.5882614	total: 9.76s	remaining: 250ms
273:	learn: 0.5882365	total: 9.79s	remaining: 214ms
274:	learn: 0.5882249	total: 9.83s	remaining: 179ms
275:	learn: 0.5882048	total: 9.87s	remaining: 143ms
276:	learn: 0.5881880	total: 9.9s	remaining: 107ms
277:	learn: 0.5881603	total: 9.94s	remaining: 71.5ms
278:	learn: 0.5881351	total: 9.97s	remaining: 35.8ms
279:	learn: 0.5881203	total: 10s	remaining: 0us
Step 16
0:	learn: 0.7612613	total: 54.4ms	remaining: 15.2s
1:	learn: 0.6749938	total: 92.2ms	remaining: 12.8s
2:	learn: 0.6421097	total: 127ms	remaining: 11.7s
3:	learn: 0.6298400	total: 165ms	remaining: 11.4s
4:	learn: 0.62

146:	learn: 0.5979780	total: 5.32s	remaining: 4.81s
147:	learn: 0.5979187	total: 5.36s	remaining: 4.78s
148:	learn: 0.5978727	total: 5.39s	remaining: 4.74s
149:	learn: 0.5978255	total: 5.43s	remaining: 4.71s
150:	learn: 0.5977912	total: 5.47s	remaining: 4.67s
151:	learn: 0.5977679	total: 5.51s	remaining: 4.64s
152:	learn: 0.5977251	total: 5.54s	remaining: 4.6s
153:	learn: 0.5976875	total: 5.58s	remaining: 4.57s
154:	learn: 0.5976468	total: 5.62s	remaining: 4.53s
155:	learn: 0.5976139	total: 5.65s	remaining: 4.49s
156:	learn: 0.5975848	total: 5.69s	remaining: 4.46s
157:	learn: 0.5975540	total: 5.72s	remaining: 4.42s
158:	learn: 0.5975246	total: 5.76s	remaining: 4.38s
159:	learn: 0.5974925	total: 5.79s	remaining: 4.34s
160:	learn: 0.5974643	total: 5.83s	remaining: 4.31s
161:	learn: 0.5974267	total: 5.87s	remaining: 4.27s
162:	learn: 0.5973968	total: 5.9s	remaining: 4.24s
163:	learn: 0.5973614	total: 5.94s	remaining: 4.2s
164:	learn: 0.5973315	total: 5.98s	remaining: 4.17s
165:	learn: 0.5

In [7]:
print("Making submission...")

y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('cat1.csv', float_format='%.4f', index=None)

Making submission...


# Using LSTMs

In [8]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras import callbacks
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [9]:
stores_items = pd.DataFrame(index=df_2017.index)
test_ids = df_test[['id']]

items = items.reindex( stores_items.index.get_level_values(1) )

X_train = X_train.values
X_test = X_test.values
X_val = X_val.values
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(.1))
model.add(Dense(32))
model.add(Dropout(.2))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer='adam', metrics=['mse'])

N_EPOCHS = 5

val_pred = []
test_pred = []
# wtpath = 'weights.hdf5'  # To save best epoch. But need Keras bug to be fixed first.
sample_weights=np.array( pd.concat([items["perishable"]] * 6) * 0.25 + 1 )
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    xv = X_val
    yv = y_val[:, i]
    model.fit(X_train, y, batch_size = 512, epochs = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv) ) 
    val_pred.append(model.predict(X_val))
    test_pred.append(model.predict(X_test))

Step 1
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
 - 7s - loss: 0.3905 - mse: 0.3685 - val_loss: 0.3172 - val_mse: 0.3172
Epoch 2/5
 - 7s - loss: 0.3516 - mse: 0.3329 - val_loss: 0.3089 - val_mse: 0.3089
Epoch 3/5
 - 7s - loss: 0.3443 - mse: 0.3262 - val_loss: 0.3003 - val_mse: 0.3003
Epoch 4/5
 - 7s - loss: 0.3408 - mse: 0.3230 - val_loss: 0.3008 - val_mse: 0.3008
Epoch 5/5
 - 7s - loss: 0.3380 - mse: 0.3205 - val_loss: 0.2973 - val_mse: 0.2973
Step 2
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
 - 7s - loss: 0.3658 - mse: 0.3451 - val_loss: 0.3310 - val_mse: 0.3310
Epoch 2/5
 - 7s - loss: 0.3622 - mse: 0.3418 - val_loss: 0.3282 - val_mse: 0.3282
Epoch 3/5
 - 7s - loss: 0.3609 - mse: 0.3406 - val_loss: 0.3272 - val_mse: 0.3272
Epoch 4/5
 - 7s - loss: 0.3603 - mse: 0.3400 - val_loss: 0.3278 - val_mse: 0.3278
Epoch 5/5
 - 7s - loss: 0.3597 - mse: 0.3395 - val_loss: 0.3263 - val_mse: 0.3263
Step 3
Train on 1005090 samples, validate on 167515 samples


Epoch 2/5
 - 7s - loss: 0.3806 - mse: 0.3617 - val_loss: 0.3512 - val_mse: 0.3512
Epoch 3/5
 - 7s - loss: 0.3789 - mse: 0.3602 - val_loss: 0.3505 - val_mse: 0.3505
Epoch 4/5
 - 7s - loss: 0.3786 - mse: 0.3600 - val_loss: 0.3496 - val_mse: 0.3496
Epoch 5/5
 - 7s - loss: 0.3783 - mse: 0.3597 - val_loss: 0.3502 - val_mse: 0.3502
Step 16
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
 - 7s - loss: 0.3963 - mse: 0.3745 - val_loss: 0.3749 - val_mse: 0.3749
Epoch 2/5
 - 7s - loss: 0.3908 - mse: 0.3693 - val_loss: 0.3767 - val_mse: 0.3767
Epoch 3/5
 - 7s - loss: 0.3895 - mse: 0.3682 - val_loss: 0.3733 - val_mse: 0.3733
Epoch 4/5
 - 7s - loss: 0.3890 - mse: 0.3677 - val_loss: 0.3750 - val_mse: 0.3750
Epoch 5/5
 - 7s - loss: 0.3885 - mse: 0.3672 - val_loss: 0.3720 - val_mse: 0.3720


In [10]:
n_public = 5 # Number of days in public test set
weights=pd.concat([items["perishable"]]) * 0.25 + 1
print("Unweighted validation mse: ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose()) )
print("Full validation mse:       ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose(), sample_weight=weights) )
print("'Public' validation mse:   ", mean_squared_error(
    y_val[:,:n_public], np.array(val_pred).squeeze(axis=2).transpose()[:,:n_public], 
    sample_weight=weights) )
print("'Private' validation mse:  ", mean_squared_error(
    y_val[:,n_public:], np.array(val_pred).squeeze(axis=2).transpose()[:,n_public:], 
    sample_weight=weights) )

Unweighted validation mse:  0.3664459796181569
Full validation mse:        0.36589094650866594
'Public' validation mse:    0.33597940877967175
'Private' validation mse:   0.3794871000218451


In [11]:
y_test = np.array(test_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_test, index=stores_items.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [12]:
submission = test_ids.join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lstm.csv', float_format='%.4f', index=None)

# Neural Networks

In [14]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM
from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import gc

In [27]:
scaler = StandardScaler()
scaler.fit(np.concatenate((X_train, X_val, X_test)))
X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
X_test[:] = scaler.transform(X_test)

X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
X_val = X_val.as_matrix()
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

ValueError: Found array with dim 3. StandardScaler expected <= 2.

In [ ]:
def build_model():
    model = Sequential()
    model.add(LSTM(512, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(.2))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(128))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(64))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(32))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(16))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(1))

    return model

In [ ]:
N_EPOCHS = 2000

val_pred = []
test_pred = []
# wtpath = 'weights.hdf5'  # To save best epoch. But need Keras bug to be fixed first.
sample_weights=np.array( pd.concat([items["perishable"]] * num_days) * 0.25 + 1 )
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    y_mean = y.mean()
    xv = X_val
    yv = y_val[:, i]
    model = build_model()
    opt = optimizers.Adam(lr=0.001)
    model.compile(loss='mse', optimizer=opt, metrics=['mse'])

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
        ]
    model.fit(X_train, y - y_mean, batch_size = 65536, epochs = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv-y_mean), callbacks=callbacks )
    val_pred.append(model.predict(X_val)+y_mean)
    test_pred.append(model.predict(X_test)+y_mean)

In [ ]:
weight = items["perishable"] * 0.25 + 1
err = (y_val - np.array(val_pred).squeeze(axis=2).transpose())**2
err = err.sum(axis=1) * weight
err = np.sqrt(err.sum() / weight.sum() / 16)
print('nwrmsle = {}'.format(err))

y_val = np.array(val_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_val, index=df_2017.index,
    columns=pd.date_range("2017-07-26", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
df_preds["unit_sales"] = np.clip(np.expm1(df_preds["unit_sales"]), 0, 1000)
df_preds.reset_index().to_csv('nn_cv.csv', index=False)

In [ ]:
print("Making submission...")
y_test = np.array(test_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('nn_sub.csv', float_format='%.4f', index=None)